In [ ]:
# For the project data sets 1 and 2

import numpy as np
from scipy.stats import norm
import math

def isString(data):
    try:
        float(data)
        return False
    except:
        pass
        return True

uniqueitems={}
categcols = []
nominalcols = []

def processData():
    filename = input("Enter filename: ")
    rawData = [line.strip().split('\t') for line in open(filename, 'r')]
    rawData = np.array(rawData)
    for i in range(len(rawData[0])-1):
        itemlist = {}
        if isString(rawData[0][i]):
            uniqueitems[i]={}
            nominalcols.append(i)
            items = list(set(rawData[:,i]))
            
            for k in range(len(items)):
                itemlist[items[k]] = k
            uniqueitems[i] = itemlist
        else:
            categcols.append(i)
        for j in range(len(rawData)):
            if(rawData[j][i] in itemlist.keys()):
                rawData[j][i] = itemlist[rawData[j][i]]
    return rawData

print(uniqueitems)



In [ ]:
data = processData().astype(float)
points= np.matrix(data[:,:-1],dtype=float,copy=False)
ground_truth_labels = np.asarray(data[:,-1],dtype=int)
dataSplit = np.array_split(points,10)
ground_truth_labels_split = np.array_split(ground_truth_labels,10)
print ("uniqueitems", uniqueitems)

for index in range(1):

    trainLabel = np.asarray(np.concatenate([x for i,x in enumerate(ground_truth_labels_split) if i != index]))
    testData = np.asarray(dataSplit[index])

    trainData = np.asarray(np.vstack([x for i,x in enumerate(dataSplit) if i != index]))
    trainDataCont = np.delete(trainData,list(uniqueitems.keys()),axis=1)

    testDataCont = np.delete(testData,list(uniqueitems.keys()),axis=1)
    testLabel = np.asarray(ground_truth_labels_split[index])

    trainNeg = trainDataCont[trainLabel == 0.0]
    trainPos = trainDataCont[trainLabel == 1.0]

    mean_neg = np.mean(trainNeg,axis=0)
    std_neg = np.std(trainNeg, axis=0)

    dist_neg = norm.pdf(testDataCont,mean_neg,std_neg)

    mean_pos = np.mean(trainPos,axis=0)
    std_pos = np.std(trainPos, axis=0)

    dist_pos =norm.pdf(testDataCont,mean_pos,std_pos)

    testNegProbs = []
    testPosProbs = []

    for i in range(len(testData)):
        testNegProbs.append(np.prod(dist_neg[i]))
        testPosProbs.append(np.prod(dist_pos[i]))

    zero_cat_list =  np.ones((len(testData)))
    one_cat_list = np.ones((len(testData)))

    for index_cat in range(len(nominalcols)):
        trainDataCat = trainData[:,nominalcols[index_cat]]
        testDataCat = testData[:,nominalcols[index_cat]]

        trainNegCat = trainDataCat[trainLabel == 0.0]
        trainPosCat = trainDataCat[trainLabel == 1.0]

        prob_of_zero = len(trainNegCat)/len(trainDataCat)

        prob_of_one = len(trainPosCat)/len(trainDataCat)
    
    for ind in range(len(testDataCat)):
        count_of_x = len(trainDataCat[trainDataCat==testDataCat[ind]])

        probX =  count_of_x/len(trainDataCat)
        probXonNeg = len(trainNegCat[np.where((trainNegCat ==testDataCat[ind]))])/len(trainNegCat)
        probXonPos = len(trainPosCat[np.where((trainPosCat ==testDataCat[ind]))])/len(trainPosCat)

        if probXonNeg == 0 or probXonNeg == 1:
            probXonNeg = (len(trainNegCat[np.where((trainNegCat ==testDataCat[ind]))]  ) + 1) /(len(trainNegCat) + len(set(trainDataCat)))


        if probXonPos == 0 or probXonPos == 1:
            probXonPos = (len(trainPosCat[np.where((trainPosCat ==testDataCat[ind]))]) + 1) /(len(trainPosCat) + len(set(trainDataCat)))
        print("probs :",probXonNeg,prob_of_zero,probX,ind, index_cat)
        probNegonX = (probXonNeg * prob_of_zero)  / probX

        probPosonX = (probXonPos * prob_of_one) / probX

        zero_cat_list[ind]=zero_cat_list[ind]*(probNegonX)

        one_cat_list[ind]=one_cat_list[ind]*(probPosonX)

    finalPreds=[]

    
    print(len(testData))
    for i in range(len(testData)):

        final_prob_zero = zero_cat_list[i]*testNegProbs[i]
        final_prob_one= one_cat_list[i]*testPosProbs[i]
        if final_prob_zero > final_prob_one:
            finalPreds.append(0)
        else:
            finalPreds.append(1)

    true_pos = 0
    false_neg =0
    false_pos=0
    true_neg = 0

    acc = 0
    prec = 0
    recall = 0
    f1_measure = 0

    print("final predict", finalPreds)
    print("testLabel", testLabel)
    print("lengths", len(finalPreds), len(testLabel))

    for i in range(len(finalPreds)):
        if testLabel[i] == 1 and finalPreds[i] == 1:
            true_pos += 1
        elif testLabel[i] == 1 and finalPreds[i] == 0:
            false_neg += 1
        elif testLabel[i] == 0 and finalPreds[i] == 1:
            false_pos += 1
        elif testLabel[i] == 0 and finalPreds[i] == 0:
            true_neg += 1
    
    acc += (float(true_pos+true_neg)/(true_pos+false_neg+false_pos+true_neg))
    
    if(true_pos+false_pos != 0):
        prec += (float(true_pos)/(true_pos+false_pos))

    if(true_pos+false_neg != 0):
        recall += (float(true_pos)/(true_pos+false_neg))
    
    f1_measure += (float(2*true_pos)/((2*true_pos)+false_neg+false_pos))

        #return acc, prec, recall, f1_measure
    print("results  (Accuracy,Precision,Recall,F1_Measure):", acc, prec, recall, f1_measure)
    print("res", true_neg,true_pos,false_neg,false_pos)


In [ ]:
# For demo inputs

import numpy as np
from scipy.stats import norm
import math

testdata = [["sunny","cool","high","weak"],["sunny","cool","high","weak"]]
finalPos = []
finalNegs = []
rawData = [line.strip().split('\t') for line in open("project3_dataset4.txt", 'r')]
rawData = np.array(rawData)

data= np.matrix(rawData[:,:-1],copy=False)
ground_truth_labels = np.asarray(rawData[:,-1],dtype=int)

for col in data[0]:
    trainNegCat = data[ground_truth_labels == 0.0]
    trainPosCat = data[ground_truth_labels == 1.0]
print(trainNegCat)
print(trainPosCat)
probNeg = len(trainNegCat)/len(data)
probPos = len(trainPosCat)/len(data)
print("probs", probNeg, probPos)
for i in range(len(testdata)):
    posProbs = 1
    negProbs = 1
    for j in range(len(testdata[i])):
        np.asarray(trainPosCat[:,j])
        countPos = len([k for k in (np.asarray(trainPosCat[:,j])) if(k == testdata[i][j])])
        countNegs = len([k for k in (np.asarray(trainNegCat[:,j])) if(k == testdata[i][j])])
        posProbs *= countPos/len(trainPosCat)
        negProbs *= countNegs/len(trainNegCat)
    print(testdata[i], ": description posterior for class0", negProbs)
    print(testdata[i], ": description posterior for class1", posProbs)
    finalPos.append(posProbs*probPos)
    finalNegs.append(negProbs*probNeg)
    print("prob for Positive Class", finalPos)
    print("prob for Negative Class", finalNegs)

    # for j in range(len(testdata[i])):
    #     np.asarray(trainPosCat[:,j])
    #     countPos = len([k for k in (np.asarray(trainPosCat[:,j])) if(k == testdata[i][j])])
    #     countNegs = len([k for k in (np.asarray(trainNegCat[:,j])) if(k == testdata[i][j])])
    #     totPos = len([k for k in (np.asarray(data[:,j])) if(k == testdata[i][j])])
    #     totalProb *= totPos/len(data)
    #     posProbs *= countPos/len(trainPosCat)
    #     negProbs *= countNegs/len(trainNegCat)
    # finalPos.append((posProbs*probPos)/totalProb)
    # finalNegs.append((negProbs*probNeg)/totalProb)
    # print("probs", finalPos, finalNegs)
